In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

from tests.sama4d.keypoint_tracking.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [44]:
scene_spec = get_cheezitbox_scene_loader()
scene = scene_spec["feature_track_data_loader"]()

In [49]:
task = KeypointTrackingTask.task_from_feature_track_data(scene, n_frames=6)

In [51]:
rr.init(f'task--{scene_spec["scene_name"]}-7')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [54]:
from tests.sama4d.keypoint_tracking.patch_tracking_solver import AdamPatchTracker
task_spec = task.get_task_specification()
solver = AdamPatchTracker()
solution = solver.solve(task_spec)

In [55]:
solver.visualize_solver_state(task_spec)

In [14]:
import jax
import jax.numpy as jnp

In [11]:
rgbds = task_spec["video"]
Xs_WC = task_spec["Xs_WC"]
X_WC = b3d.Pose.identity()
centers = task_spec["initial_keypoint_positions_2D"]
r = task_spec["renderer"]
fx, fy, cx, cy = r.fx, r.fy, r.cx, r.cy
depths = rgbds[..., 3]
xyzs_C = b3d.utils.xyz_from_depth_vectorized(depths, fx, fy, cx, cy)
xyzs_W = X_WC.apply(xyzs_C)

In [12]:
pairwise_euclidean_dists = jnp.linalg.norm(centers[:, None] - centers[None], axis=-1)
min_nonzero_dist = jnp.min(jnp.where(pairwise_euclidean_dists != 0, pairwise_euclidean_dists, jnp.inf))
del_pix = jnp.astype(jnp.ceil(min_nonzero_dist / (2 * jnp.sqrt(2))), int)
del_pix = max(del_pix, 2)

In [20]:
center = centers[0]
center = jnp.astype(jnp.round(center), int)
center_x, center_y = center[0], center[1]
patch_points_C = jax.lax.dynamic_slice(xyzs_C[0], (center_x-del_pix,center_y-del_pix,0), (2*del_pix-1,2*del_pix-1,3)).reshape(-1,3)
num_nonzero = jnp.sum(jnp.where(patch_points_C[...,2] != 0, 1, 0))
mean_position_nonzero = jnp.sum(patch_points_C, axis=0) / num_nonzero

Array(36, dtype=int32)

In [ ]:
middle_index = patch_points_C.shape[0] // 2 + 1
middle_point = patch_points_C[middle_index]
patch_points_C_nonzero = jnp.where(
    (patch_points_C[..., 2] != 0)[:, None],
    patch_points_C,
    middle_point
)
patch_points_C_nonzero